# Amazonレビューデータの可視化

収集したデータの特徴を分析し、可視化します。

In [5]:
%pip install japanize-matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import japanize_matplotlib

# スタイル設定
plt.style.use('seaborn-v0_8')  # seabornスタイルの正しい指定方法
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['font.size'] = 12



[notice] A new release of pip is available: 24.2 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
# データ読み込み
df = pd.read_csv('../data/processed/electronics_reviews.csv')
print(f"データサイズ: {len(df)}行")
df.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/processed/electronics_reviews.csv'

## 1. 基本統計量の可視化

In [ ]:
# レビューの評価分布
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='overall')
plt.title('レビュー評価の分布')
plt.xlabel('評価（星の数）')
plt.ylabel('レビュー数')
plt.show()

In [ ]:
# レビューの長さ分布
df['review_length'] = df['reviewText'].str.split().str.len()

plt.figure(figsize=(12, 6))
sns.histplot(data=df, x='review_length', bins=50)
plt.title('レビューの長さ分布')
plt.xlabel('単語数')
plt.ylabel('頻度')
plt.show()

# 基本統計量
print("\nレビュー長の基本統計量:")
print(df['review_length'].describe())

## 2. 時系列分析

In [ ]:
# Unix時間をdatetime形式に変換
df['review_date'] = pd.to_datetime(df['unixReviewTime'], unit='s')

# 月ごとのレビュー数の推移
monthly_reviews = df.resample('M', on='review_date').size()

plt.figure(figsize=(15, 6))
monthly_reviews.plot()
plt.title('月別レビュー投稿数の推移')
plt.xlabel('日付')
plt.ylabel('レビュー数')
plt.grid(True)
plt.show()

## 3. 評価と長さの関係分析

In [ ]:
# 評価ごとの平均レビュー長
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='overall', y='review_length')
plt.title('評価別のレビュー長分布')
plt.xlabel('評価（星の数）')
plt.ylabel('レビューの長さ（単語数）')
plt.show()

## 4. テキスト特徴の分析

In [ ]:
from collections import Counter
import re

def get_word_freq(texts):
    words = ' '.join(texts).lower()
    words = re.findall(r'\w+', words)
    return Counter(words)

# 評価別の頻出単語
for rating in df['overall'].unique():
    reviews = df[df['overall'] == rating]['reviewText']
    word_freq = get_word_freq(reviews)
    
    print(f"\n評価 {rating} 星の頻出単語:")
    for word, count in word_freq.most_common(10):
        print(f"{word}: {count}回")

## 5. レビューの要約統計

In [ ]:
summary_stats = {
    '総レビュー数': len(df),
    '平均評価': df['overall'].mean(),
    '中央値評価': df['overall'].median(),
    '平均レビュー長': df['review_length'].mean(),
    '最長レビュー': df['review_length'].max(),
    '最短レビュー': df['review_length'].min(),
    '5星評価の割合': (df['overall'] == 5).mean() * 100,
    '1星評価の割合': (df['overall'] == 1).mean() * 100
}

print("\nデータセットの要約統計:")
for key, value in summary_stats.items():
    print(f"{key}: {value:.2f}")